# GeneGan Jupyter Barebone

#### Overview: Barebone Jupyter notebook version of vanilla-GAN.
#### Note: Need to activate genomelake environment before running this code. Simply type 'genomelake' in terminal on kali under 'jesikmin'.

In [1]:
%env CUDA_VISIBLE_DEVICES=0
import os, sys
sys.path.append("..")
# custom file path package
from data import Data_Directories
# custom utility package
from utils.compute_util import *
# package for genomic data
from pybedtools import Interval, BedTool
from genomelake.extractors import ArrayExtractor, BigwigExtractor
# package for plotting
import matplotlib.pyplot as plt
%matplotlib inline
# package for correlation
from scipy.stats.stats import pearsonr,spearmanr
# tensorflow
import tensorflow as tf

env: CUDA_VISIBLE_DEVICES=0


Using TensorFlow backend.


In [11]:
# Arguments
window_size = 2001
sample_num = 10000
day = 'day0'
frag = '140'
histone = 'H3K27ac'

In [12]:
does_load_model = True
model_path = ""
if does_load_model:
    model_path = "models/cnn_2000_new/best_model.h5"

In [13]:
# # Logging directories
# model_dir = os.path.join("models", "gan_fixed_nonsmooth_nonwide")
# srv_dir = os.path.join("/srv", "www", "kundaje", "jesikmin", "experiments", "gan_fixed_nonsmooth_nonwide")
# if not os.path.exists(model_dir):
#     os.makedirs(model_dir)
# if not os.path.exists(srv_dir):
#     os.makedirs(srv_dir)

In [14]:
# Train/val/test intervals
DATA_DIR = '/srv/scratch/jesikmin'
train_dir, val_dir, test_dir = os.path.join(DATA_DIR, 'train_interval'),\
                               os.path.join(DATA_DIR, 'val_interval'),\
                               os.path.join(DATA_DIR, 'test_interval')

In [15]:
print train_dir, val_dir, test_dir

/srv/scratch/jesikmin/train_interval /srv/scratch/jesikmin/val_interval /srv/scratch/jesikmin/test_interval


In [16]:
# Get train/val/test intervals
train_intervals = list(BedTool(train_dir))
val_intervals = list(BedTool(val_dir))
test_intervals = list(BedTool(test_dir))
print '# of Train Intervals: {}'.format(len(train_intervals))
print '# of Val Intervals: {}'.format(len(val_intervals))
print '# of Test Intervals: {}'.format(len(test_intervals))

# of Train Intervals: 106855
# of Val Intervals: 35618
# of Test Intervals: 37068


In [17]:
# Get input/output data directories
data = Data_Directories()
print data.intervals.keys()
print data.input_atac[day].keys()
print data.output_histone[day].keys()

['day6', 'day3', 'day0']
['100', '140']
['H3K27me3', 'H3K4me1', 'H3K27ac']


#### Extract input candidates

In [20]:
# Create an ArrayExtractor for ATAC-seq of a given day and specified fragment length
input_candidates = ArrayExtractor(data.input_atac[day][frag])
print 'Finished extracting bigwig for {}, {}bp'.format(day, frag)

Finished extracting bigwig for day0, 140bp


#### Extract output candiates

In [21]:
# Create a BigWigExtractor for histone mark of a given day
output_candidates = BigwigExtractor(data.output_histone[day][histone])
print 'Finished extracting bigwig for {}, {}'.format(day, histone)

Finished extracting bigwig for day0, H3K27ac


In [22]:
# Normalize train intervals
normalized_train_intervals = [normalize_interval(interval, window_size) for interval in train_intervals if normalize_interval(interval, window_size)]
print 'Finished normalizing train intervals!'
# Normalize val intervals
normalized_val_intervals = [normalize_interval(interval, window_size) for interval in val_intervals if normalize_interval(interval, window_size)]
print 'Finished normalizing val intervals!'
# Normalize test intervals
normalized_test_intervals = [normalize_interval(interval, window_size) for interval in test_intervals if normalize_interval(interval, window_size)]
print 'Finished normalizing test intervals!'

Finished normalizing train intervals!
Finished normalizing val intervals!
Finished normalizing test intervals!


In [9]:
# Assertions of normalization step
assert (len(train_intervals)==len(normalized_train_intervals))
assert (len(val_intervals)==len(normalized_val_intervals))
assert (len(test_intervals)==len(normalized_test_intervals))
# Examples of normalized intervals
print "Examples of original train intervals"
print [(int(_interval.start)+int(_interval[-1]), [int(_interval.start), int(_interval.end)])
       for _interval in train_intervals[:3]]
print "Examples of normalized train intervals with window size of {}".format(window_size)
print [([int(_interval.start), int(_interval.end)])
       for _interval in  normalized_train_intervals[:3]]

Examples of original intervals
[(123412027, [123411855, 123412989]), (123411941, [123411855, 123412989]), (131908564, [131908487, 131910071])]
Examples of normalized intervals with window size of 2001
[[123411027, 123413028], [123410941, 123412942], [131907564, 131909565]]


In [25]:
X_train = input_candidates(normalized_train_intervals)
print "Finished fetching X_train"
X_val = input_candidates(normalized_val_intervals)
print "Finished fetching X_val"
X_test = input_candidates(normalized_test_intervals)
print "Finished fetching X_test"
print X_train.shape, X_val.shape, X_test.shape

Finished fetching X_train
Finished fetching X_val
Finished fetching X_test
(106852, 2001, 5) (35618, 2001, 5) (37068, 2001, 5)


In [28]:
print "Dimension of ATAC-seq signal (input): {}".format(X_train[0].shape)

Dimension of ATAC-seq signal (input): (2001, 5)


In [29]:
# Replace nan values with zeros
y_train = np.nan_to_num(output_candidates(normalized_train_intervals))
print "Finished fetching y_train"
y_val = np.nan_to_num(output_candidates(normalized_val_intervals))
print "Finished fetching y_val"
y_test = np.nan_to_num(output_candidates(normalized_test_intervals))
print "Finished fetching y_test"
print y_train.shape, y_val.shape, y_test.shape

Finished fetching y_train
Finished fetching y_val
Finished fetching y_test
(106852, 2001) (35618, 2001) (37068, 2001)


In [30]:
y_train = np.expand_dims(y_train, axis=2)
y_val = np.expand_dims(y_val, axis=2)
y_test = np.expand_dims(y_test, axis=2)
print y_train.shape, y_val.shape, y_test.shape

(106852, 2001, 1) (35618, 2001, 1) (37068, 2001, 1)


In [31]:
print "Dimension of histone mark signal (output): {}".format(y_train[0].shape)

Dimension of histone mark signal (output): (2001, 1)


In [32]:
# Import keras
from keras.layers import AveragePooling1D, Input, Dense, Conv1D, Dropout, BatchNormalization, Activation, ZeroPadding1D, Reshape, Flatten
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential, Model
from keras import optimizers
from keras import metrics
from keras import losses
from keras import backend as K
from keras.callbacks import Callback, TensorBoard, ReduceLROnPlateau, ModelCheckpoint
from keras.optimizers import Adam, SGD

In [33]:
'''
HYPERPARAMETERS
'''
# GAN Discriminator
smooth_rate = 0.1
d_train_freq = 16
# Dropout Rate
dropout_rate = 0.5
# First conv layer
hidden_filters_1 = 32
hidden_kernel_size_1 = window_size
# Second conv layer
output_filters = 1
output_kernel_size = 32
# Training
batch_size = 128
num_epochs = 200
evaluation_freq = 10

In [34]:
# Helper functions for writing the scores into bigwig file
from itertools import izip
from itertools import groupby
import subprocess

def interval_key(interval):
    return (interval.chrom, interval.start, interval.stop)

def merged_scores(scores, intervals, merge_type):
    # A generator that returns merged intervals/scores
    # Scores should have shape: #examples x #categories x #interval_size
    # Second dimension can be omitted for a 1D signal
    signal_dims = scores.ndim - 1
    assert signal_dims in {1, 2}

    # Only support max for now
    assert merge_type == 'max'
    score_first_dim = 1 if signal_dims == 1 else scores.shape[1]

    dtype = scores.dtype

    sort_idx, sorted_intervals = \
        zip(*sorted(enumerate(intervals),
                    key=lambda item: interval_key(item[1])))
    sorted_intervals = BedTool(sorted_intervals)

    # Require at least 1bp overlap
    # Explicitly convert to list otherwise it will keep opening a file when
    # retrieving an index resulting in an error (too many open files)
    interval_clust = list(sorted_intervals.cluster(d=-1))
    for _, group in groupby(izip(sort_idx, interval_clust),
                            key=lambda item: item[1].fields[-1]):
        idx_interval_pairs = list(group)
        group_idx, group_intervals = zip(*idx_interval_pairs)

        if len(idx_interval_pairs) == 1:
            yield group_intervals[0], scores[group_idx[0], ...]
        else:
            group_chrom = group_intervals[0].chrom
            group_start = min(interval.start for interval in group_intervals)
            group_stop = max(interval.stop for interval in group_intervals)

            # This part needs to change to support more merge_types (e.g. mean)
            group_score = np.full((score_first_dim, group_stop - group_start),
                                  -np.inf, dtype)
            for idx, interval in idx_interval_pairs:
                slice_start = interval.start - group_start
                slice_stop = slice_start + (interval.stop - interval.start)
                group_score[..., slice_start:slice_stop] = \
                    np.maximum(group_score[..., slice_start:slice_stop],
                               scores[idx, ...])
            if signal_dims == 1:
                group_score = group_score.squeeze(axis=0)
            yield Interval(group_chrom, group_start, group_stop), group_score
            
def interval_score_pairs(intervals, scores, merge_type):
    return (izip(intervals, scores) if merge_type is None
            else merged_scores(scores, intervals, merge_type))

def _write_1D_deeplift_track(scores, intervals, file_prefix, merge_type='max',
                             CHROM_SIZES='/mnt/data/annotations/by_release/hg19.GRCh37/hg19.chrom.sizes'):
    assert scores.ndim == 2

    bedgraph = file_prefix + '.bedGraph'
    bigwig = file_prefix + '.bw'

    print 'Writing 1D track of shape: {}'.format(scores.shape)
    print 'Writing to file: {}'.format(bigwig)

    with open(bedgraph, 'w') as fp:
        for interval, score in interval_score_pairs(intervals, scores,
                                                    merge_type):
            chrom = interval.chrom
            start = interval.start
            for score_idx, val in enumerate(score):
                fp.write('%s\t%d\t%d\t%g\n' % (chrom,
                                               start + score_idx,
                                               start + score_idx + 1,
                                               val))
    print 'Wrote bedgraph.'

    try:
        output = subprocess.check_output(
            ['wigToBigWig', bedgraph, CHROM_SIZES, bigwig],
            stderr=subprocess.STDOUT)
        print 'wigToBigWig output: {}'.format(output)
    except subprocess.CalledProcessError as e:
        print 'wigToBigWig terminated with exit code {}'.format(
            e.returncode)
        print 'output was:\n' + e.output

    print 'Wrote bigwig.'

In [36]:
# Our GAN Model (based on vanilla gan)
class GAN():
    def __init__(self, window_size, sample_num,
                 X_train, y_train,
                 X_val, y_val,
                 X_test, y_test,
                 model_dir, srv_dir):
        
        # Set train/val/test
        self.X_train, self.y_train = X_train, y_train
        self.X_val, self.y_val = X_val, y_val
        self.X_test, self.y_test = X_test, y_test
        
        self.model_dir = model_dir
        self.srv_dir = srv_dir
        
        # Basic parameters
        # 1) Window size of input/ouput signal
        self.window_size = window_size
        # 2) Number of channels
        self.channels = 5
        # 3) Input and Output shape
        self.input_shape = (self.window_size, self.channels,)
        self.output_shape = (self.window_size, 1,)
        # 4) Total number of samples
        self.sample_num = sample_num
        
        # Adam optimizer for generator
        optimizer = Adam(0.002, beta_1=0.5)
        # Adam optimizer for discriminator
        doptimizer = Adam(0.0005, beta_1=0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy', 
                                   optimizer=doptimizer,
                                   metrics=['accuracy'])

        # Build and compile the generator
        self.generator = self.build_generator()
        self.generator.compile(loss='binary_crossentropy',
                               optimizer=optimizer)

        # The generator takes noise as input and generated imgs
        z = Input(shape=self.input_shape)
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The valid takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator) takes
        # noise as input => generates images => determines validity 
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy',
                              optimizer=optimizer)
        print "Combined Model"
        print self.combined.summary()

    def build_generator(self):
        # Generator
        # 1) 32 * window_size Conv1D layers with RELU and Dropout

        noise_shape = self.input_shape
        
        model = Sequential()

        model.add(Conv1D(hidden_filters_1,
                         hidden_kernel_size_1,
                         padding="same",
                         strides=1,
                         input_shape=noise_shape,
                         activation='relu',
                         name='gen_conv1d_1'))
        model.add(Dropout(dropout_rate,
                  name='gen_dropout_1'))

        # 2) 1 * 16 Conv1D layers with Linear
        # NOTE: All same padding
        model.add(Conv1D(output_filters,
                         output_kernel_size,
                         padding='same',
                         strides=1,
                         activation='linear',
                         name='gen_conv1d_output'))
        
        print "Generator"
        model.summary()

        noise = Input(shape=noise_shape)
        img = model(noise)
        
        # load weights for generator if specified
        if does_load_model:
            print "-"*50
            print model.get_weights()
            model.load_weights(model_path, by_name=True)
            print "-"*50
            print model.get_weights()

        return Model(noise, img)

    def build_discriminator(self):
        # Discriminator
        # 1) 16 * 200 Conv1D with LeakyRelu, Dropout, and BatchNorm
        model = Sequential()
        
        model.add(Conv1D(hidden_filters_1,
                         200,
                         padding="valid",
                         strides=1,
                         input_shape=self.output_shape))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(dropout_rate))
        model.add(BatchNormalization(momentum=0.8))
        
        # 2) Average Pooling, Flatten, Dense, and LeakyRelu
        model.add(AveragePooling1D(25))
        model.add(Flatten())
        model.add(Dense(int(window_size/16)))
        model.add(LeakyReLU(alpha=0.2)) 
        
        # 3) Final output with sigmoid
        model.add(Dense(1, activation='sigmoid'))
        
        print "Discriminator"
        model.summary()

        img = Input(shape=self.output_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size):
        
        max_pearson = -1.0
        
        # size of the half of the batch
        half_batch = int(batch_size / 2)
            
        d_loss_real, d_loss_fake, g_loss = [1, 0], [1, 0], [1, 0]
            
        for epoch in range(epochs):
            
            # list for storing losses/accuracies for both discriminator and generator
            d_losses, d_accuracies, g_losses = [], [], []
            
            # sufficient number of minibatches for each epoch
            for _minibatch_idx in range(128):

                # ---------------------
                #  Train Discriminator
                # ---------------------

                # Select a random half batch of images
                dis_idx = np.random.randint(0, y_train.shape[0], half_batch)
                # Get the real images
                imgs = y_train[dis_idx]

                # Generate a half batch of new images
                dis_noise = self.X_train[dis_idx]
                gen_imgs = self.generator.predict(dis_noise)
                
                # Train the discriminator with label smoothing
                if smooth_rate > 0.0:
                    smoothed_idx = np.random.choice(half_batch, int(half_batch*smooth_rate), replace=False)
                smoothed_labels = np.ones((half_batch, 1))
                if smooth_rate > 0.0:
                    smoothed_labels[smoothed_idx] = 0
                
                # Train the discriminator for every d_train_freq minibatch
                if _minibatch_idx % d_train_freq == 0:
                    # 1) Concatenate the real images and generated images
                    all_imgs = np.concatenate([imgs, gen_imgs])
                    # 2) Concatenate the real labels and labels for generated images
                    all_labels = np.concatenate([smoothed_labels, np.zeros((half_batch, 1))])
                    # 3) Shuffle two lists in same order
                    def unison_shuffled_copies(a, b):
                        assert len(a) == len(b)
                        p = np.random.permutation(len(a))
                        return a[p], b[p]
                    all_imgs, all_labels = unison_shuffled_copies(all_imgs, all_labels)
                    # 4) Train the discriminator for current batch
                    d_loss = self.discriminator.train_on_batch(all_imgs, all_labels)
                                    
                    # ---------------------
                    # Store loss and accuracy
                    # ---------------------
                    d_losses.append(d_loss[0])
                    d_accuracies.append(d_loss[1])
                
                # ---------------------
                #  Train Generator
                # ---------------------
                
                # 1) Randomly choose the batch number of indices
                gen_idx = np.random.randint(0, y_train.shape[0], batch_size)
                # 2) Get the randomly chosen inputs (noises in traditional gan)
                gen_noise = self.X_train[gen_idx]
                # 3) Create the labels for noises
                # The generator wants the discriminator to label the generated samples as valid (ones)
                valid_y = np.array([1] * batch_size)

                # 4) Train the generator
                # Discriminator is no longer trained at this point
                # Generator do its best to fake discriminator
                g_loss = self.combined.train_on_batch(gen_noise, valid_y)

                g_losses.append(g_loss)
            
            # ---------------------
            # Convert each histories into numpy arrays to get means
            # ---------------------
            d_losses = np.array(d_losses)
            d_accuracies = np.array(d_accuracies)
            g_losses = np.array(g_losses)
            
            # ---------------------
            # Get generator's prediction and compute overall pearson on train set
            # ---------------------
            predictions = self.generator.predict(self.X_train).flatten()
            avg_pearson = pearsonr(predictions, self.y_train.flatten())
            print "Pearson R on Train set: {}".format(avg_pearson)
            
            # ---------------------
            # Get generator's prediction and compute overall pearson on validation set
            # ---------------------
            val_predictions = self.generator.predict(self.X_val).flatten()
            avg_val_pearson = pearsonr(val_predictions, self.y_val.flatten())
            print "Pearson R on Val set: {}".format(avg_val_pearson)
            
            # if current pearson on validation set is greatest so far, update the max pearson,
            # and 
            if max_pearson < avg_val_pearson:
                print "Perason on val improved from {} to {}".format(max_pearson, avg_val_pearson)
                _write_1D_deeplift_track(predictions.reshape(self.X_train.shape[0], self.window_size),
                                         normalized_train_intervals, os.path.join(self.srv_dir, 'train'))
                _write_1D_deeplift_track(val_predictions.reshape(self.X_val.shape[0], self.window_size),
                                         normalized_val_intervals, os.path.join(self.srv_dir, 'val'))
                f = open(os.path.join(self.srv_dir, 'meta.txt'), 'wb')
                f.write(str(epoch) + " " + str(avg_pearson) + "  " + str(avg_val_pearson) + "\n")
                max_pearson = avg_val_pearson
                
                # ---------------------
                # Get generator's prediction and compute overall pearson on test set
                # ---------------------
                test_predictions = self.generator.predict(self.X_test).flatten()
                avg_test_pearson = pearsonr(test_predictions, self.y_test.flatten())
                print "Pearson R on Test set: {}".format(avg_test_pearson)
                f.write("Test Pearson: " + str(avg_test_pearson))
                f.close()
                _write_1D_deeplift_track(val_predictions.reshape(self.X_test.shape[0], self.window_size),
                                         normalized_test_intervals, os.path.join(self.srv_dir, 'test'))
                
                self.generator.save(os.path.join(self.model_dir, 'best_generator.h5'))
                self.discriminator.save(os.path.join(self.model_dir, 'best_discriminator.h5'))
                
            
            # Print the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_losses.mean(), 100.0*d_accuracies.mean(), g_losses.mean()))

In [37]:
# Helper function for computing Pearson R in Keras
def pearson(y_true, y_pred):
    x = y_true
    y = y_pred
    mx = K.mean(x)
    my = K.mean(y)
    xm, ym = x-mx, y-my
    r_num = K.sum(np.multiply(xm,ym))
    r_den = K.sqrt(np.multiply(K.sum(K.square(xm)), K.sum(K.square(ym))))
    r = r_num / r_den
    r = K.maximum(K.minimum(r, 1.0), -1.0)
    return K.square(r)

In [28]:
print "Training the model..."
gan = GAN(window_size, sample_num,
          X_train, y_train,
          X_val, y_val,
          X_test, y_test,
          model_dir, srv_dir)
gan.train(num_epochs, batch_size)

Fitting the model...
Discriminator
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_3 (Conv1D)            (None, 1802, 32)          6432      
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 1802, 32)          0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 1802, 32)          0         
_________________________________________________________________
batch_normalization_3 (Batch (None, 1802, 32)          128       
_________________________________________________________________
average_pooling1d_3 (Average (None, 72, 32)            0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 2304)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 125) 

Combined Model
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         (None, 2001, 5)           0         
_________________________________________________________________
model_8 (Model)              (None, 2001, 1)           321217    
_________________________________________________________________
model_7 (Model)              (None, 1)                 294811    
Total params: 616,028
Trainable params: 321,217
Non-trainable params: 294,811
_________________________________________________________________
None
Pearson R on Train set: (-0.13246292, 0.0)
Pearson R on Val set: (-0.17326175, 0.0)
Perason on val improved from -1.0 to (-0.17326175, 0.0)
Writing 1D track of shape: (8000, 2001)
Writing to file: /srv/www/kundaje/jesikmin/gan_fixed_nonsmooth_nonwide/train.bw
Wrote bedgraph.
wigToBigWig output: 
Wrote bigwig.
Writing 1D track of shape: (2000, 2001)
Writing to file: /srv/w

Pearson R on Train set: (0.1396088, 0.0)
Pearson R on Val set: (0.17076859, 0.0)
43 [D loss: 0.202737, acc.: 95.21%] [G loss: 0.999713]
Pearson R on Train set: (0.195499, 0.0)
Pearson R on Val set: (0.21353495, 0.0)
44 [D loss: 0.271274, acc.: 90.92%] [G loss: 1.282908]
Pearson R on Train set: (0.29523858, 0.0)
Pearson R on Val set: (0.30124798, 0.0)
45 [D loss: 0.214921, acc.: 92.38%] [G loss: 1.194173]
Pearson R on Train set: (0.26748991, 0.0)
Pearson R on Val set: (0.27928147, 0.0)
46 [D loss: 0.156534, acc.: 95.31%] [G loss: 1.083248]
Pearson R on Train set: (0.14695144, 0.0)
Pearson R on Val set: (0.18316913, 0.0)
47 [D loss: 0.207688, acc.: 93.65%] [G loss: 1.100943]
Pearson R on Train set: (0.27692011, 0.0)
Pearson R on Val set: (0.28581348, 0.0)
48 [D loss: 0.174338, acc.: 94.24%] [G loss: 0.990468]
Pearson R on Train set: (0.19383903, 0.0)
Pearson R on Val set: (0.21937162, 0.0)
49 [D loss: 0.145149, acc.: 96.00%] [G loss: 1.052385]
Pearson R on Train set: (0.28886703, 0.0)
Pe

Pearson R on Train set: (0.28099322, 0.0)
Pearson R on Val set: (0.29857999, 0.0)
97 [D loss: 0.136533, acc.: 95.02%] [G loss: 1.206554]
Pearson R on Train set: (0.23145241, 0.0)
Pearson R on Val set: (0.26925838, 0.0)
98 [D loss: 0.103111, acc.: 97.17%] [G loss: 1.116656]
Pearson R on Train set: (0.28716421, 0.0)
Pearson R on Val set: (0.31599602, 0.0)
99 [D loss: 0.127253, acc.: 97.36%] [G loss: 1.145701]
Pearson R on Train set: (0.18772438, 0.0)
Pearson R on Val set: (0.22872466, 0.0)
100 [D loss: 0.133544, acc.: 95.31%] [G loss: 1.171169]
Pearson R on Train set: (0.29617894, 0.0)
Pearson R on Val set: (0.31164742, 0.0)
101 [D loss: 0.107348, acc.: 96.88%] [G loss: 1.143002]
Pearson R on Train set: (0.28545457, 0.0)
Pearson R on Val set: (0.29324779, 0.0)
102 [D loss: 0.107114, acc.: 96.78%] [G loss: 1.124478]
Pearson R on Train set: (0.28678155, 0.0)
Pearson R on Val set: (0.30647355, 0.0)
103 [D loss: 0.102826, acc.: 97.27%] [G loss: 1.154449]
Pearson R on Train set: (0.24593537, 

Pearson R on Train set: (0.31954449, 0.0)
Pearson R on Val set: (0.32387361, 0.0)
157 [D loss: 0.053244, acc.: 98.63%] [G loss: 1.107920]
Pearson R on Train set: (0.29301196, 0.0)
Pearson R on Val set: (0.31703591, 0.0)
158 [D loss: 0.085173, acc.: 97.95%] [G loss: 1.115534]
Pearson R on Train set: (0.31845382, 0.0)
Pearson R on Val set: (0.32678843, 0.0)
159 [D loss: 0.066304, acc.: 97.95%] [G loss: 1.156746]
Pearson R on Train set: (0.15471406, 0.0)
Pearson R on Val set: (0.17156518, 0.0)
160 [D loss: 0.141930, acc.: 96.29%] [G loss: 1.485949]
Pearson R on Train set: (0.23136602, 0.0)
Pearson R on Val set: (0.25710094, 0.0)
161 [D loss: 0.042237, acc.: 98.83%] [G loss: 1.561953]
Pearson R on Train set: (0.25757244, 0.0)
Pearson R on Val set: (0.269086, 0.0)
162 [D loss: 0.054789, acc.: 98.34%] [G loss: 1.511140]
Pearson R on Train set: (0.25952366, 0.0)
Pearson R on Val set: (0.27359396, 0.0)
163 [D loss: 0.019356, acc.: 99.41%] [G loss: 1.469830]
Pearson R on Train set: (0.25164753,

KeyboardInterrupt: 